In [1]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from mpl_toolkits.axisartist.axislines import SubplotZero
from pylab import text
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)


In [2]:
stmt = 'SELECT DATE_FORMAT(date, \'%Y-%m\') as date, COUNT(1) AS tweets FROM social.machine_learning_only group by DATE_FORMAT(date, \'%Y-%m\') ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC' 
stmt2 = 'SELECT DATE_FORMAT(date, \'%Y-%m\') as date, COUNT(1) AS tweets FROM social.machine_learning group by DATE_FORMAT(date, \'%Y-%m\') ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC' 
df = pd.read_sql(stmt, con=engine)
df2 = pd.read_sql(stmt2, con=engine)

# Machine Learning

In [3]:
#ML
year = ['2010-01-01', '2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']
papers = [6151, 6572, 7207, 7937, 9308, 11141, 13609, 17449, 23410, 35663, 59378, 70504, 90290]

df['date']  = pd.to_datetime(df['date'])
df2['date']  = pd.to_datetime(df['date'])
year = pd.to_datetime(year)

In [4]:
#ML plot
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 3]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Number of Records')
plt.plot(df['date'], df['tweets'], c = '#E31B23', linewidth = 1, label='Filtered Tweets')
plt.plot(df2['date'], df2['tweets'], c = 'black', linewidth = 1, label='Unfiltered Tweets')
plt.plot(year, papers, c = '#339898', linewidth = 1, label='Academic Publications')
plt.legend()
# plt.savefig(os.path.join(path, 'ml_data.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [5]:
degree = 2
y = np.array(df2['tweets'].to_list())
X = np.array(df2['date'].to_list())
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

In [6]:
df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.X, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()

# results.summary()

In [7]:
column = df_trend["y"]
max_index = column.idxmax()
df_trend_1 = df_trend.head(max_index)
tail_index =  len(df_trend.index) - max_index
df_trend_2 = df_trend.tail(tail_index)

In [8]:
#first half
df_trend = df_trend_1
z = np.polyfit(df_trend.X, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()
# print(results.summary().as_latex())

In [9]:
#first half plot
ax = plt.gca()
plt.xlabel('Time')
plt.ylabel('Number of Records')
plt.plot(df_trend.d, df_trend.y, label='Unfiltered Tweets', c = 'black')
plt.plot(df_trend.d, model(df_trend.X), "--", c = '#E31B23', label = 'Exponential Trend Function')
plt.xticks(rotation = 45)
# myFmt = mdates.DateFormatter('%Y')
# ax.xaxis.set_major_formatter(myFmt)
plt.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [5, 5]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.savefig(os.path.join(path, 'ml_trend_1a.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()


In [10]:
#second half
df_trend = df_trend_2
z = np.polyfit(df_trend.X, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()
# print(results.summary().as_latex())

In [11]:
#second half plot
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [5, 5]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Number of Records')
plt.plot(df_trend.d, df_trend.y, label='Unfiltered Tweets', c = 'black')
plt.plot(df_trend.d, model(df_trend.X), "--", c = '#E31B23', label = 'Exponential Trend Function')
plt.xticks(rotation = 45)
pos = ['2018-01-01','2019-01-01','2020-01-01','2021-01-01','2022-01-01']
l = [2018,2019,2020,2021,2022]
ax.set(xticks=pos, xticklabels=l)
plt.legend()
plt.savefig(os.path.join(path, 'ml_trend_2a.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

## Filtered tweets

In [12]:
degree = 2
y = np.array(df['tweets'].to_list())
X = np.array(df['date'].to_list())
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

In [13]:
df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.X, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()

# results.summary()

In [14]:
column = df_trend["y"]
max_index = column.idxmax()
df_trend_1 = df_trend.head(max_index)
tail_index =  len(df_trend.index) - max_index
df_trend_2 = df_trend.tail(tail_index)

In [15]:
#first half
df_trend = df_trend_1
z = np.polyfit(df_trend.X, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()
# print(results.summary().as_latex())

In [16]:
#first half plot
ax = plt.gca()
plt.xlabel('Time')
plt.ylabel('Number of Records')
plt.plot(df_trend.d, df_trend.y, label='Filtered Tweets', c = 'black')
plt.plot(df_trend.d, model(df_trend.X), "--", c = '#E31B23', label = 'Exponential Trend Function')
plt.xticks(rotation = 45)

plt.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [5, 5]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.savefig(os.path.join(path, 'ml_trend_1b.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

In [17]:
#second half
df_trend = df_trend_2
z = np.polyfit(df_trend.X, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()
# print(results.summary().as_latex())

In [18]:
#second half plot
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [5, 5]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300
plt.xlabel('Time')
plt.ylabel('Number of Records')
plt.plot(df_trend.d, df_trend.y, label='Filtered Tweets', c = 'black')
plt.plot(df_trend.d, model(df_trend.X), "--", c = '#E31B23', label = 'Exponential Trend Function')
plt.xticks(rotation = 45)
pos = ['2018-01-01','2019-01-01','2020-01-01','2021-01-01','2022-01-01']
l = [2018,2019,2020,2021,2022]
ax.set(xticks=pos, xticklabels=l)
plt.legend()
plt.savefig(os.path.join(path, 'ml_trend_2b.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

# 5g

In [19]:
#5G

# stmt = 'SELECT DATE_FORMAT(date, \'%Y-%m\') as date, COUNT(1) AS tweets FROM social.five_g_only group by DATE_FORMAT(date, \'%Y-%m\') ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC' 
# stmt2 = 'SELECT DATE_FORMAT(date, \'%Y-%m\') as date, COUNT(1) AS tweets FROM social.five_g group by DATE_FORMAT(date, \'%Y-%m\') ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC' 
# df = pd.read_sql(stmt, con=engine)
# df2 = pd.read_sql(stmt2, con=engine)

In [20]:
#5g
# year = ['2010-01-01', '2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']
# papers = [471, 585, 882, 921, 874, 1365, 2024, 3220, 4732, 6856, 9074, 9668, 11971]

# df['date']  = pd.to_datetime(df['date'])
# df2['date']  = pd.to_datetime(df['date'])
# year = pd.to_datetime(year)

In [21]:
#5g plot
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.rcParams["figure.figsize"] = [8, 3]
# plt.rcParams.update({'font.size': 12})
# plt.rcParams['figure.dpi'] = 300
# plt.xlabel('Time')
# plt.ylabel('Number of Records')
# plt.plot(df['date'], df['tweets'], c = '#E31B23', linewidth = 1, label='Filtered Tweets')
# plt.plot(df2['date'], df2['tweets'], c = 'black', linewidth = 1, label='Unfiltered Tweets')
# plt.plot(year, papers, c = '#339898', linewidth = 1, label='Academic Publications')
# plt.legend()
# plt.savefig(os.path.join(path, '5g_data.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()

In [22]:
#5g plot
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.rcParams["figure.figsize"] = [8, 3]
# plt.rcParams.update({'font.size': 12})
# plt.rcParams['figure.dpi'] = 300
# plt.xlabel('Time')
# plt.ylabel('Number of Records')
# plt.plot(year, papers, c = '#339898', linewidth = 1, label='Academic Publications')
# plt.savefig(os.path.join(path, '5g_aca_data.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()

# Blockchain

In [23]:
#blockchain

# stmt = 'SELECT DATE_FORMAT(date, \'%Y-%m\') as date, COUNT(1) AS tweets FROM social.blockchain_only group by DATE_FORMAT(date, \'%Y-%m\') ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC' 
# stmt2 = 'SELECT DATE_FORMAT(date, \'%Y-%m\') as date, COUNT(1) AS tweets FROM social.blockchain group by DATE_FORMAT(date, \'%Y-%m\') ORDER BY DATE_FORMAT(date, \'%Y-%m\') ASC' 
# df = pd.read_sql(stmt, con=engine)
# df2 = pd.read_sql(stmt2, con=engine)

In [24]:
#blockchain
# year = ['2010-01-01', '2010-12-31', '2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31']
# papers = [0, 2, 0, 1, 2, 12, 41, 182, 823, 3018, 6025, 7964, 9885]

# df['date']  = pd.to_datetime(df['date'])
# df2['date']  = pd.to_datetime(df['date'])
# year = pd.to_datetime(year)

In [25]:
#blockchain plot
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.rcParams["figure.figsize"] = [8, 3]
# plt.rcParams.update({'font.size': 12})
# plt.rcParams['figure.dpi'] = 300
# plt.xlabel('Time')
# plt.ylabel('Number of Records')
# plt.plot(df['date'], df['tweets'], c = '#E31B23', linewidth = 1, label='Filtered Tweets')
# plt.plot(df2['date'], df2['tweets'], c = 'black', linewidth = 1, label='Unfiltered Tweets')
# plt.plot(year, papers, c = '#339898', linewidth = 1, label='Academic Publications')
# plt.legend()
# plt.savefig(os.path.join(path, '5g_data.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()

In [26]:
#blockchain plot
# ax = plt.gca()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# plt.rcParams["figure.figsize"] = [8, 3]
# plt.rcParams.update({'font.size': 12})
# plt.rcParams['figure.dpi'] = 300
# plt.xlabel('Time')
# plt.ylabel('Number of Records')
# plt.plot(year, papers, c = '#339898', linewidth = 1, label='Academic Publications')
# plt.savefig(os.path.join(path, 'blockchain_aca_data.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
# plt.close()